In [1]:
from ldn.utils import submit_job, get_job_status, get_cloudwatch_logs
import time

In [ ]:
job_name = "version-0-1-0-job"
job_queue = "normalQueue"
job_definition = "auspatious-ldn"
container_overrides = {
    "command": [
        "ldn-processor",
        # "--tile",
        # "Ref::tile",
        "--year",
        "Ref::year",
        "--version",
        "Ref::version",
        "--n-workers",
        "Ref::n_workers",
        "--threads-per-worker",
        "Ref::threads_per_worker",
        "--memory-limit",
        "Ref::memory_limit",
        "Ref::overwrite",
    ],
    "vcpus": 16,
    "memory": 122880,
}
parameters = {
    "tile": "238,47",
    "year": "2023",
    "version": "0.1.0",
    "n_workers": "4",
    "threads_per_worker": "32",
    "memory_limit": "80GB",
    "overwrite": "--overwrite",
}

job_id = submit_job(
    job_name, job_queue, job_definition, container_overrides, parameters, multi=True
)
print("Job submitted with id:", job_id)

old_logs = set()
status = get_job_status(job_id)

# Get status of job every 10 seconds
while status not in('SUCCEEDED', 'FAILED'):
    new_status = get_job_status(job_id)
    if new_status != status:
        print('Job status:', status)
        status = new_status

    if new_status == 'RUNNING':
        logs = get_cloudwatch_logs(job_id)

        for log in logs:
            log_message = log['message']
            if log_message not in old_logs:
                print(log_message)
                old_logs.add(log_message)

    time.sleep(10)
else:
    print(f'Job {status} successfully')

Job submitted with id: b4b15695-9b69-4746-9de4-d2f034659408
Job status: SUBMITTED
